$$\dot{M} = \int \rho v_r \Delta S $$

In [20]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
cd ~/gscratch/athena_2/mri_hi_res/

/global/scratch2/sd/dorislee/athena_2/mri_hi_res


In [ ]:
import sys
sys.path.remove('/global/homes/d/dorislee/.local/lib/python2.7/site-packages/yt-3.2.dev0-py2.7-linux-x86_64.egg')
sys.path.remove('/global/homes/d/dorislee/.local/lib/python2.7/site-packages')
sys.path.append('/global/homes/d/dorislee/mypythonpackages')

In [28]:
tstep_filename  = "mri_hires_smaller.out1.00085.ath"

In [30]:
import h5py
import glob
debug =True
if debug: print "Working on {}".format(tstep_filename) 
hdf5 = h5py.File(tstep_filename, 'r+')
NUM_MESHBLOCK=96
D = np.zeros(128)

Working on mri_hires_smaller.out1.00085.ath


In [31]:
meshblock =hdf5["MeshBlock9"]

192x128x128 $\rightarrow 32^3 \rightarrow$ each meshblock is 6x4x4

In [42]:
debug =True
if debug: print "Working on {}".format(tstep_filename) 
hdf5 = h5py.File(tstep_filename, 'r+')
NUM_MESHBLOCK=96
D = np.zeros(128)
for M in np.arange(NUM_MESHBLOCK):
#     print "Working on MeshBlock {}".format(M)
    meshblock =hdf5["MeshBlock{}".format(M)]
    density= meshblock["rho"].value#density for each meshblock
    N_r = density.shape[2]
    N_theta=density.shape[1]
    N_phi= density.shape[0]
    x1f = meshblock["x1f"].value
    x2f = meshblock["x2f"].value
    x3f = meshblock["x3f"].value
    vr = meshblock["vel1"].value
    loc = meshblock.attrs["LogicalLocation"]
#     for rii in np.arange(N_r):
#         phi_i =n+32*loc[2]
        #Compute D_i in each meshblock
#         if rii = 0
    rii=0 # we only want the mass flux onto the inner most cell
    val=0 #val=integrand to sum over rho*vr*dS
    ri = (x1f[rii]+x1f[rii+1])*0.5
    dri = abs(x1f[rii]-x1f[rii+1])
    for ti in np.arange(N_theta):
        tii =ti+32*loc[2]
        thetai = (x2f[ti]+x2f[ti+1])*0.5
        dthetai = abs(x2f[ti]-x2f[ti+1])
        for pii in np.arange(N_phi):
            dphii=abs(x3f[ti]-x3f[ti+1])
            dS= ri*sin(thetai)*dri*dphii
            val = val +density[pii,ti,rii]*vr[pii,ti,rii]*dS
        D[tii]=D[tii]+val
print D

Working on mri_hires_smaller.out1.00085.ath
[ -9.41336870e-06   1.01867471e-06   3.72160361e-05   8.72201934e-05
   1.42749099e-04   2.21731475e-04   2.99646495e-04   3.47575016e-04
   3.58893399e-04   3.57794111e-04   3.70101307e-04   3.97841264e-04
   4.14626282e-04   4.24458251e-04   4.49410874e-04   4.82165467e-04
   4.86371715e-04   4.68899167e-04   4.47929838e-04   4.33837609e-04
   4.45169467e-04   4.98986155e-04   5.64795845e-04   6.27318377e-04
   6.98681345e-04   7.78609883e-04   8.44980820e-04   8.90435093e-04
   9.23903097e-04   9.44618088e-04   9.51175588e-04   9.46285730e-04
   6.82867770e-05   9.22793934e-05   5.05255397e-05  -4.71571536e-05
  -1.63619571e-04  -2.96485601e-04  -3.99281257e-04  -4.64597623e-04
  -5.18702403e-04  -5.67537841e-04  -6.08653751e-04  -6.52858756e-04
  -6.77083206e-04  -6.66775540e-04  -6.38360389e-04  -6.07054536e-04
  -5.78272870e-04  -5.50382507e-04  -5.19310832e-04  -4.95574863e-04
  -4.88123056e-04  -5.09589167e-04  -5.70113866e-04  -6.649

In [46]:
print "Mass accretion rate: ", sum(D)

Mass accretion rate:  -0.0592378218406
